In [6]:
import speech_recognition as sr
import nltk
from nltk.corpus import wordnet as wn
import socket
import struct
import re

In [3]:
betSyn = []
for syn in wn.synsets("bet"):
    for i in syn.lemmas():
        betSyn.append(i.name())
for syn in wn.synsets("do"):
    for i in syn.lemmas():
        betSyn.append(i.name())
for syn in wn.synsets("put"):
    for i in syn.lemmas():
        betSyn.append(i.name())
for syn in wn.synsets("go"):
    for i in syn.lemmas():
        betSyn.append(i.name())
betSynSet=set(betSyn)      
betSyn=list(betSynSet)

underSyn = []
for syn in wn.synsets("under"):
    for i in syn.lemmas():
        underSyn.append(i.name())
for syn in wn.synsets("less"):
    for i in syn.lemmas():
        underSyn.append(i.name())
underSynSet=set(underSyn)
underSyn=list(underSynSet)

overSyn=[]
for syn in wn.synsets("over"):
    for i in syn.lemmas():
        overSyn.append(i.name())
for syn in wn.synsets("more"):
    for i in syn.lemmas():
        overSyn.append(i.name())
overSynSet=set(overSyn)
overSyn=list(overSynSet)

exactSyn=[]
for syn in wn.synsets("exactly"):
    for i in syn.lemmas():
        exactSyn.append(i.name())
exactSynSet=set(exactSyn)
exactSyn=list(exactSynSet)


In [4]:
def getCommand():
    r = sr.Recognizer()

    with sr.Microphone() as source:
        # read the audio data from the default microphone
        audio_data = r.record(source, duration=5)
        print("Recognizing...")
        # convert speech to text
        text = r.recognize_google(audio_data)
    
    t2 = text.replace('$','')
    
    t3 = re.sub(r'(?<=\d)(,)(?=\d)','',t2)
    
    print(t3)
    
    tokens = nltk.word_tokenize(t3)
        
    tagged = nltk.pos_tag(tokens)
    entities = nltk.chunk.ne_chunk(tagged)

    numList = []

    for i,word in enumerate(tokens):
        if word.isdigit():
            numList.append(i)

    betFound = False
    actionFound = False
    bet=-1
    action=-1
            
    for i in numList:
        if tokens[i-1] in exactSyn:
            actionFound=True
            action=1
            continue
        elif tokens[i-1] in underSyn:
            actionFound=True
            action=0
            continue
        elif tokens[i-1] in overSyn:
            actionFound=True
            action=2
            continue
        elif tokens[i-1] in betSyn:
            betFound=True
            bet=int(tokens[i])
            continue
    if not actionFound:
        tokenSet=set(tokens)
        if not tokenSet.isdisjoint(underSynSet):
            actionFound=True
            action=0
        elif not tokenSet.isdisjoint(overSynSet):
            actionFound=True
            action=2
        elif not tokenSet.isdisjoint(exactSynSet):
            actionFound=True
            action=1
        elif '7' in tokens:
            actionFound = True
            action = 1
    print(bet)
    print(action)
    
    return [bet,action]
    
    

In [5]:
print(betSyn)
print(underSyn)
print(overSyn)
print(exactSyn)

['cause', 'get_going', 'pop_off', 'hold_out', 'calculate', 'blend', 'go_bad', 'buy_the_farm', 'behave', 'arrange', 'pass', 'wager', 'Doctor_of_Osteopathy', 'get_along', 'brawl', 'lay', 'Adam', 'stake', 'sound', 'become', 'turn', 'redact', 'ecstasy', 'hold_up', 'assign', 'bet', 'endure', 'pass_away', 'reckon', 'make', 'XTC', 'X', 'doh', 'fail', 'play', 'function', 'DO', 'last', 'practice', 'expire', 'break_down', 'exercise', 'plump', 'tour', "cash_in_one's_chips", 'lead', 'depart', 'go_away', 'crack', 'decease', 'coiffure', 'manage', 'suffice', 'start', 'position', 'belong', 'go_game', 'snuff_it', 'set', 'conk', 'put_option', 'coiffe', 'cast', 'fling', 'make_out', 'serve', 'conk_out', 'run_short', 'place', 'bash', 'disco_biscuit', 'give_way', 'whirl', 'operate', 'execute', 'do', 'invest', 'fit', 'count', 'spell', 'blend_in', 'live_on', 'come', 'survive', 'offer', 'rifle', 'work', 'live', 'die', 'give_out', 'exit', 'look', 'coif', 'pose', 'order', 'hug_drug', 'perish', 'locomote', 'run',

In [6]:
serve = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serve.bind(('localhost',4149))

serving = True

serve.listen(5)


while serving:
    try:
        (fromUnity,fromAdd) = serve.accept()
        recv = fromUnity.recv(4)
    except socket.timeout:
        serving=False
        serve.close()
        break
    print(int.from_bytes(recv,'little'))
    
    command = getCommand()
    
    toUnity = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        toUnity.connect(("localhost", 4150))
        toUnity.send(struct.pack("@ii",command[0],command[1]))
        print("sent")
    except socket.timeout:
        serving=False
        toUnity.close()
        serve.close()
        

1
Recognizing...
Tibet 100 on over
-1
2
sent
1
Recognizing...
too bad 100 on over
-1
2
sent
1
Recognizing...
I'd like to put 100 on over
100
2
sent
1
Recognizing...
let's do 700 on over
-1
2
sent
1
Recognizing...
I'll do 400 on 7
400
1
sent
1
Recognizing...
let's do 400 on
400
-1
sent
1
Recognizing...
let's do 400 on 7
400
1
sent
1
Recognizing...
let's do a thousand on over
-1
2
sent
1
Recognizing...
let's do 1,000 on over
-1
2
sent
